In [15]:
#네이버 view검색을 활용한 블로그 검색

In [30]:
# 라이브러리 import
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options

In [17]:
#크롤링 시작과 끝 날짜(월별로 크롤링)
first_days = pd.date_range('2022/01/01','2022/12/07', freq='MS') # 각 달의 첫 날
last_days = pd.date_range('2022/01/01','2022/12/07', freq='M') #각 달의 마지막날

keyword_list = ['태연인스타그램'] #검색 키워드 추가

In [31]:
#크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('disable-dex-shm-usage') # overcome limited resource porcess

chrome_path='chromedriver.exe' # 현재 컴퓨터의 크롬드라이버 위치

In [19]:
# 블로그 주소 수집

blog_url_list = []

#네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
        
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 1000 #
    
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭

    #날짜 지정
    for k in range(len(first_days)):
        try:
            #시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()            
            
            
            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            
            scroll = (url_want/30)-1 #한 페이지에 기본으로 30개 글이 나옴
            
            # 스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                time.sleep(random.uniform(1,1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                
                if new_height == last_height:
                    break
                    
                last_height = new_height
                
                
            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a',class_='api_txt_lines total_tit')
            
            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) #주소 수집
                
            print('{0} 키워드 {1}년{2}월'.format(keyword, 
                                            first_days.year[k], first_days.month[k]), 
                  len(blog_url),'개 블로그 url 주소 수집')
            
            
            driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass
            

C:\Users\user\AppData\Local\Temp\ipykernel_896\304093927.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


태연인스타그램 키워드 2022년1월 109 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년2월 89 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년3월 110 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년4월 86 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년5월 97 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년6월 66 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년7월 76 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년8월 123 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년9월 99 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년10월 58 개 블로그 url 주소 수집
태연인스타그램 키워드 2022년11월 85 개 블로그 url 주소 수집


In [20]:
len(blog_url_list)

998

In [21]:
#수집 후 url list 중복 확인

real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)


998

In [22]:
k = pd.DataFrame(blog_url_list)
k.to_csv('태연인스타그램_url_list(2022).csv', encoding='utf-8-sig')

In [33]:
# 블로그 크롤링 불러오기
url = pd.read_csv('제니인스타그램_url_list(2022).csv')
blog_url_list = list(url['0']) #리스트 형태로 저장

In [34]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list[8555:]: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        try:
            driver.get(url) #url 하나씩 글을 수집할꺼야
            time.sleep(1)

            try:
                # 전체 본문 가지고 와서
                driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
                time.sleep(1)

                soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
                postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

                reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
                postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집

                try:
                    timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우

                except:
                    timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
                timeline = timeline.text
                time_list.append(timeline)
                review_list.append(postview_reg)
                url_list.append(url)
                time.sleep(random.uniform(1,1.6))
            except UnexpectedAlertPresentException:
                pass
        except:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%50 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\user\AppData\Local\Temp\ipykernel_896\1251833440.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


50개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://wfmania2002.tistory.com/1741
네이버 블로그 아닌 url:  https://moamusic.tistory.com/entry/%EC%9C%8C-%EC%8A%A4%EB%AF%B8%EC%8A%A4-%EC%8B%9C%EC%83%81%EC%8B%9D-%EC%A4%91-%ED%8F%AD%ED%96%89-%EC%9D%B4%EC%9C%A0%EB%8A%94-%EB%88%84%EA%B5%AC-%EC%9E%98%EB%AA%BB%EC%9D%BC%EA%B9%8C
네이버 블로그 아닌 url:  https://storyhealing.tistory.com/963
네이버 블로그 아닌 url:  https://kant10.tistory.com/entry/%EB%B8%94%EB%9E%99%ED%95%91%ED%81%AC-%EB%A6%AC%EC%82%AC-%EC%95%84%EC%B0%94-%EC%9B%90%ED%94%BC%EC%8A%A4-%EC%9E%85%EA%B3%A0-%EC%84%B9%EC%8B%9C%E7%BE%8E-%EB%B0%9C%EC%82%B0-%ED%8C%8C%EB%A6%AC-%EB%92%A4%ED%9D%94%EB%93%A4%EB%A7%8C-%EC%82%AC%EC%A7%84
네이버 블로그 아닌 url:  https://dayuk0919.tistory.com/440
100개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://truegirlsave2.tistory.com/2231
네이버 블로그 아닌 url:  https://truegirlsave2.tistory.com/2312
네이버 블로그 아닌 url:  https://truegirlsave2.tistory.com/3038
네이버 블로그 아닌 url:  https://chueok7.tistory.com/entry/%EB%B8%94%EB%9E%99%ED%95%91%ED%81%AC-%EC%A0%9C%EB%8B%88-%EC%9D%B8%

In [25]:
len(time_list)

962

In [26]:
data = {'time':time_list, 
       'review':review_list,
       'url':url_list}

df = pd.DataFrame(data)

print('크롤링 소요시간: {}초'.format(time.time()-start))
print('총 {}개 블로그 크롤링 완료'.format(count))

크롤링 소요시간: 4544.696233510971초
총 962개 블로그 크롤링 완료


In [27]:
df

,time,review,url
0,2022. 10. 7. 20:20,"[연예계] 태연 인스타스토리 베네피트,NERD 태연화보 ...",https://blog.naver.com/gnssudpdy/222894509235
1,2022. 10. 3. 16:43,[블챌] 평범한 일상 기록하기 '숲속의 담'과 함께한 휴일 ...,https://blog.naver.com/goonu1205/222890658036
2,2022. 3. 5. 15:36,게시판 손질도 간편하고 누구나 쉽게 소화할 수 있는 여자...,https://blog.naver.com/rmrmdkdk1/222664567952
3,2022. 2. 4. 12:41,"에디터 J 여름쿨톤 태연, 김태리 겨울쿨톤 현아, 선미처...",https://blog.naver.com/iwedding_story/22263862...
4,2022. 3. 9. 23:42,탱스타그램 220309 태연 인스타그램(태연 생일) ...,https://blog.naver.com/ntjeyj/222668319273
...,...,...,...
957,2022. 1. 18. 15:10,"Fashion 태연 트레이닝복 세트, 널디 크롭후드집업 여...",https://blog.naver.com/hug387/222624812698
958,2022. 1. 16. 16:32,"패션 Y2K패션, 하이틴 패션에 대하여 Maquin...",https://blog.naver.com/maquina86/222623175570
959,2022. 5. 26. 15:32,2022 220526 태연 인스타그램 스토리 ☀️🌿 ...,https://blog.naver.com/wjdtjdrbs123/222747528438
960,2022. 4. 9. 13:57,명함제작 명찰제작 리플렛디자인 중국비자 중국상용비자 브로셔디자인 전단...,https://blog.naver.com/xanthenelson/222695996087


In [28]:
df.to_csv('제니인스타그램(2022).csv',encoding='utf-8-sig')